In [ ]:
import os
from pathlib import Path
import pickle
from fastfusion import Specification
from fastfusion.mapper.FFM._make_pmappings.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM._join_pmappings.simexplore import join_sims
from fastfusion.mapper.FFM._make_pmappings.mapping_filter_tags import get_one_split_tag, get_ffmt_tag


ARCH_DIR          = Path('architecture/')
WORKLOAD_DIR      = Path('workloads/')
MAPPINGS_SIMS_DIR = Path('results/sims/')
MAPPINGS_DATA_DIR = Path('results/data/')


def one_split_tagger(compatibility):
    return get_one_split_tag(compatibility, "MainMemory")


NAME_TO_TAGGER = {
    'one_split': one_split_tagger
}


def get_experiment_name(tagger_name, arch_name: list[str], workload_name):
    combined_arch_name = '+'.join(arch_name)
    return f'matmul8_mixed.{tagger_name}.{combined_arch_name}.{workload_name}'


def get_sims_with_cache(tagger_name=None,
                        refresh_cache=False,
                        arch_name: list[str]=['snowcat'],
                        workload_name='matmuls8_mixed'):
    data_name  = get_experiment_name(tagger_name, arch_name, workload_name)
    result_pickle_name = MAPPINGS_DATA_DIR / f'{data_name}.pkl'
    if result_pickle_name.is_file() and not refresh_cache:
        with open(result_pickle_name, 'rb') as f:
            mappings = pickle.load(f)
            print(f'Loaded final results from cache {result_pickle_name}')
            return mappings

    if tagger_name is None:
        tagger = None
    else:
        tagger = NAME_TO_TAGGER[tagger_name]

    all_sims = []
    for a in arch_name:
        sims_name = get_experiment_name(tagger_name, [a], workload_name)
        sims_pickle_name = MAPPINGS_SIMS_DIR / f'{sims_name}.pkl'
        if sims_pickle_name.is_file() and not refresh_cache:
            with open(sims_pickle_name, 'rb') as f:
                sims = pickle.load(f)
                print(f'Loaded SIMs from {sims_pickle_name}')
                all_sims.append(sims)
                continue

        spec = Specification.from_yaml(
            ARCH_DIR / f'{a}.arch.yaml',
            WORKLOAD_DIR / f'{workload_name}.workload.yaml'
        )
        spec.calculate_component_energy_area()
        workload = spec.workload
        renames = spec.renames
        flattened_architecture = spec.get_flattened_architecture()
        sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger)

        with open(sims_pickle_name, 'wb') as f:
            pickle.dump(sims, f)

        all_sims.append(sims)

    combined_sims = {}
    for sims in all_sims:
        for einsum, sims_for_einsum in sims.items():
            if einsum not in combined_sims:
                combined_sims[einsum] = []
            combined_sims[einsum].extend(sims_for_einsum)

    spec = Specification.from_yaml(
        ARCH_DIR / f'{a}.arch.yaml',
        WORKLOAD_DIR / f'{workload_name}.workload.yaml'
    )
    flattened_architecture = spec.get_flattened_architecture()
    mappings = join_sims(combined_sims, spec, flattened_architecture, drop_valid_reservations=False)
    mappings.decompress(decompress_data)

    with open(result_pickle_name, 'wb') as f:
        pickle.dump(mappings, f)
        print(f'Saved results to cache {result_pickle_name}')

    return mappings

mappings_looptree = get_sims_with_cache('one_split', workload_name='mha_full', refresh_cache=True)
mappings_full = get_sims_with_cache(workload_name='mha_full', refresh_cache=True)
mappings_tileflow = get_sims_with_cache(tagger_name='one_split', arch_name=['snowcat_even'], workload_name='mha_full', refresh_cache=True)
mappings_ffmt = get_sims_with_cache(tagger_name='one_split', arch_name=['snowcat_ffmt'], workload_name='mha_full', refresh_cache=True)

INFO        Loading yaml file architecture/snowcat.arch.yaml
INFO        Found top key variables in architecture/snowcat.arch.yaml
INFO        Found top key architecture in architecture/snowcat.arch.yaml
INFO        Loading yaml file workloads/mha_full.workload.yaml
INFO        Found top key workload in workloads/mha_full.workload.yaml
WARNING     Loading configuration file from /home/gilbertm/work/infrastructure/work-venv/fastfusion/config.yaml
INFO        Calculated "0.5" = 0.5.
Generating storage and loop choices for Einsum I: 3it [00:00, 1397.48it/s]
Generating storage and loop choices for Einsum V: 46it [00:00, 2971.25it/s]
Generating storage and loop choices for Einsum K: 46it [00:00, 2942.16it/s]
Generating storage and loop choices for Einsum Q: 46it [00:00, 3145.28it/s]
Generating storage and loop choices for Einsum QK: 136it [00:00, 4016.09it/s]
Generating storage and loop choices for Einsum AV: 136it [00:00, 4087.25it/s]
Generating storage and loop choices for Einsum Z: 56it 

SIM I tensors: {'I'}
SIM V tensors: {'V', 'I'}
SIM K tensors: {'I', 'K'}
SIM Q tensors: {'Q', 'I'}
SIM QK tensors: {'Q', 'K', 'QK'}
SIM AV tensors: {'V', 'AV', 'QK'}
SIM Z tensors: {'AV', 'Z'}
SIM FFA tensors: {'FFA', 'Z'}
SIM FFB tensors: {'FFA'}


Grouping Partial Mappings: 100%|████████████| 335/335 [00:00<00:00, 1904.89it/s]


Initial consolidate and group: 72.10 seconds

Einsum V (2/9)
Consolidating: 0.00 seconds


Grouping Partial Mappings: 100%|████████████| 288/288 [00:00<00:00, 1921.45it/s]


...

Merging mappings K <--> Q: 100%|███████████| 4020/4020 [00:04<00:00, 947.73it/s]


Mapping merging: 4.64 seconds
Scaled runtime by 1.0. Runtime: 1.01
	Combining 18367(6525) x 18367(6525) -> 4020
	Number of groups for Einsum Q: 4020
	Number of mappings for Einsum Q: 40945
	Mappings per group for Einsum Q: 10.185323383084578

Einsum QK (5/9)
Consolidating: 0.21 seconds


Grouping Partial Mappings: 100%|████████████| 862/862 [00:00<00:00, 1356.38it/s]


Combining: 1.11 seconds
Grouping: 0.00 seconds
Bucket merging: 0.67 seconds
Removed 42152/44336 (4.93% remaining)
Removing mappings that can't be combined later: 0.27 seconds


Merging mappings Q <--> QK:  69%|██████▉   | 1504/2184 [00:02<00:00, 684.25it/s]

In [3]:
import copy
import re
from fastfusion.frontend.mapping import Iteration, Mapping, Nested, Split, Storage
from fastfusion.visualization.interactive import plotly_show
from fastfusion.mapper.FFM.deprecate_maybe.visualization import make_mapping
from fastfusion.frontend.workload import Workload
workload = Workload.from_yaml('workloads/matmuls8_mixed.workload.yaml')

plotly_show(mappings_full.data, "RESOURCE_GlobalBuffer_LEVEL_0", "Total_Energy", logscales=True, einsum_names=workload.einsum_names)

INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [4]:
plotly_show(mappings_looptree.data, "RESOURCE_GlobalBuffer_LEVEL_0", "Total_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [5]:

plotly_show(mappings_tileflow.data, "RESOURCE_GlobalBuffer_LEVEL_0", "Total_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [4]:
plotly_show(mappings_ffmt.data, "RESOURCE_GlobalBuffer_LEVEL_0", "Total_Energy", logscales=True, einsum_names=workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…